# Explore signal/price movement relationship

This is an example notebook to explore whether a trading signal results to profitable trades.

- 3 days 
- It explores a small set of pairs on Uni v3 on Polygon
- Both long and short are considered (though shorts might be theoretical only, if such a lending market doest not exist in the point of time)

First run to prepare the dataset:

```shell
python scripts/prepare-polygon-momentum-candles.py
```

In [ ]:
import datetime

import pandas as pd

from tradingstrategy.client import Client
from tradingstrategy.chain import ChainId
from tradingstrategy.pair import PandasPairUniverse
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.execution_context import notebook_execution_context
from tradeexecutor.utils.default_strategies import get_default_strategies_path
from tradeexecutor.strategy.strategy_module import read_strategy_module
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_partial_data, load_trading_and_lending_data
from tradeexecutor.strategy.universe_model import UniverseOptions

client = Client.create_jupyter_client()

# Load preprocessed candle dataset
# See prepare-polygon-momentum-candles.py   
chain_id = ChainId.polygon
time_bucket = TimeBucket.h1
exchange_slugs = ["uniswap-v3", "quickswap"]
slug_str = "-and-".join(exchange_slugs)
fpath = f"/tmp/{chain_id.get_slug()}-{slug_str}-candles-{time_bucket.value}.parquet"
all_candles_df = pd.read_parquet(fpath)

# Filter out pair ids that belong to our target dataset
exchanges = client.fetch_exchange_universe()
exchange_ids = [exchanges.get_by_chain_and_slug(ChainId.polygon, s).exchange_id for s in exchange_slugs]
pairs_df = client.fetch_pair_universe().to_pandas()
pairs_df = pairs_df.loc[pairs_df["exchange_id"].isin(exchange_ids)]


### Interactive viewer mode

Set Plotly chart output mode to interactive viewing.

In [ ]:
from tradeexecutor.utils.notebook import OutputMode, setup_charting_and_output

#setup_charting_and_output(OutputMode.interactive, min_rows=10, max_rows=35)
setup_charting_and_output(
    OutputMode.static, 
    image_format="png",
    width=1500,
    height=1200,
    max_rows=30,    
    min_rows=10,
)

#pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,.2f}'.format


## Filter trading pairs

In [ ]:
from tradingstrategy.pair import filter_for_stablecoins, StablecoinFilteringMode

good_quote_tokes = {"USDC", "WMATIC"}

# Remove pairs with expensive 1% fee tier
# Remove stable-stable pairs
tradeable_pairs_df = pairs_df.loc[pairs_df["fee"] <= 30]
tradeable_pairs_df = filter_for_stablecoins(tradeable_pairs_df, StablecoinFilteringMode.only_volatile_pairs)
tradeable_pairs_df = tradeable_pairs_df.loc[tradeable_pairs_df["quote_token_symbol"].isin(good_quote_tokes)]

# Denormalise pair ticker symbols on candles DataFrame 
tradeable_pairs_df["ticker"] = tradeable_pairs_df.apply(lambda r: f"{r.base_token_symbol}-{r.quote_token_symbol}-{r.fee}", axis=1)

pair_universe = PandasPairUniverse(tradeable_pairs_df, exchange_universe=exchanges)

# Narrow down candle data to pairs that are left after filtering
eligible_pair_ids = tradeable_pairs_df["pair_id"]
candles_df = all_candles_df.loc[all_candles_df["pair_id"].isin(eligible_pair_ids)].copy()

print(f"We have {len(tradeable_pairs_df)} tradeable pairs our of {len(pairs_df)} total loaded pairs")
print(f"We have {len(candles_df):,} candles for tradeable pairs, out of {len(all_candles_df):,} total candles")

# Denormalise pair data on candles dataframe
# https://stackoverflow.com/questions/62133801/copy-a-column-value-from-another-dataframe-based-on-a-condition
candles_df["timestamp"] = candles_df.index  # Index is lost in merge?
candles_df = candles_df.merge(tradeable_pairs_df[["pair_id", "ticker", "exchange_slug"]], on="pair_id", how="left")
candles_df = candles_df.set_index("timestamp")

## Explore signal vs price change

Create a function `calculate_signal_vs_profit` which calculates 
- Signal (naive momentum)
- Profit: the last and the best future price we can get
- Allows us to play around with different time windows
- Split between shorts and longs


In [ ]:
from pandas.tseries.frequencies import to_offset
import humanize

# Set up parameters of time windows
# we use for this notebook run
#lookback_window = pd.Timedelta(days=2)
#profit_window = pd.Timedelta(days=2)


lookback_window = pd.Timedelta(hours=24)
profit_window = pd.Timedelta(hours=24)

long_lookback_window = lookback_window * 4
daily_volume_threshold = 50_000

# short - long EMA pairs
short_long_ema_pairs = [
    (8, 24),
    (16, 48),
    (32, 96),
]

# Normalise EMA diff to 90 days/90 hours
ema_diff_normalisation_period = 90

min_entries = 14

min_age = pd.DateOffset(days=20)

quantile_sanity_threshold = 0.9995


# Use later in chart titles
signal_window_label = humanize.naturaldelta(lookback_window)
profit_window_label = humanize.naturaldelta(profit_window)




Create a function `calculate_signal_vs_price_for_pair` which calculates 
- Calculates the signal vs. for certain trading pair
- Bundle few different pairs to the same `DataFrame` so we can examine them together


In [ ]:
import numpy as np
from pandas.core.groupby import DataFrameGroupBy
from pandas_ta import ema, pvt, obv

from tradingstrategy.pair import DEXPair


def calculate_ema_signal(df: pd.DataFrame):
    """Calculate different EMA, their diffs as a single signal

    Momentum and trend following trading strategies for currencies and bitcoin
    by Janick Rohrbach, Silvan Suremann, Joerg Osterriede]
    """
    for idx, ema_tuple in enumerate(short_long_ema_pairs, start=1):
        short_ema, long_ema = ema_tuple
        df[f"ema_long_{idx}"] = ema(df["close"], length=long_ema) 
        df[f"ema_short_{idx}"] = ema(df["close"], length=short_ema)
        df[f"ema_diff_{idx}"] = df[f"ema_short_{idx}"] - df[f"ema_long_{idx}"]

        # Normalise EMA diff with three months moving standard deviation        
        df[f"ema_diff_{idx}_normalised"] = df[f"ema_diff_{idx}"] / df[f"ema_diff_{idx}"].rolling(ema_diff).std()

    return df    


def calculate_signal_vs_profit(
    df: pd.DataFrame, 
    pair: DEXPair,
    momentum_window: pd.Timedelta, 
    profit_window: pd.Timedelta,
    time_frame: pd.Timedelta,        
) -> pd.DataFrame:
    """Calculate signals and profits for all incoming candles."""

    number_of_look_back_candles = lookback_window / time_frame
    number_of_look_forward_candles = profit_window / time_frame
    assert number_of_look_forward_candles > 0 and number_of_look_forward_candles.is_integer(), f"Could not calculate candle count that fits into profit window {profit_window} for data time frame {time_frame}"
    assert number_of_look_back_candles > 0 and number_of_look_forward_candles.is_integer(), f"Could not calculate candle count that fits into profit window {lookback_window} for data time frame {time_frame}"
    number_of_look_forward_candles = int(number_of_look_forward_candles)
    number_of_look_back_candles = int(number_of_look_back_candles)

    # Create entries for past price to be used for signal
    # and future price (used for the price correlation)
    momentum_offset = to_offset(lookback_window)
    profit_offset = to_offset(profit_window)

    # No data left after filtering
    if len(df.index) == 0:
        return pd.DataFrame()

    # Calculate trading pair age in a column
    start = df.index[0]

    # Remove first N days of trading history to filter out scam pump and dumps
    df = df.loc[df.index > start + min_age].copy()

    # No data left after filtering
    if len(df) < number_of_look_back_candles:
        return pd.DataFrame()
    
    df["age"] = df.index - start

    # Fix missing prices
    df["open"] = df["open"].replace(0, np.NaN)

    df["prev"] = df["open"].shift(number_of_look_back_candles)
    df["next"] = df["open"].shift(-number_of_look_forward_candles)

    # What is our predicted price
    df["price_diff"] = (df["next"] - df["open"]) / df["open"]  # Get the profit on the final day of profit window

    # Calculate signal from the past and price difference to the future
    df["momentum"] = (df["prev"] - df["open"]) / df["open"]

    #df["shifted_close"] = df["volume"].shift(1).rolling(obv_len).sum()
    #shifted_close = df.rolling(obv_len)
    #assert shifted_close["close"] is not None
    #assert shifted_close["volume"] is not None
    #import ipdb ; ipdb.set_trace()
    df["pvt"] = pvt(df["close"], df["volume"])
    df["obv"] = obv(df["close"], df["volume"])

    df = calculate_ema_signal(df)

    #df["obv"] = obv(shifted_close["close"], shifted_close["volume"])
    #shifted = shifted.iloc[-obv_len:-1]
    # df["obv"] = obv(shifted["close"], shifted["volume"])

    # Drop any momentum value that seems to be incorrect (more than 99% price movement)
    df["momentum"] = np.where(df["momentum"] > 0.99, 0, df["momentum"])
    df["momentum"] = np.where(df["momentum"] < -0.99, 0, df["momentum"])
    
    # df.loc[df["bullish"] & (df["momentum"] >= 0), "signal"] = df["momentum"]
    # df.loc[df["bearish"] & (df["momentum"] < 0), "signal"] = df["momentum"]    
    df["rolling_cum_volume"] = df["volume"].rolling(window=long_lookback_window).sum() 
    df["rolling_obv"] = df["obv"] - df["obv"].shift(periods=number_of_look_back_candles)
    df["rolling_pvt"] = df["pvt"] - df["pvt"].shift(periods=number_of_look_back_candles)
    # df["signal"] = df["rolling_pvt"].shift(1) / df["rolling_cum_volume"].shift(1)
    #df["signal"] = df["rolling_obv"].shift(1) / df["rolling_cum_volume"].shift(1)

    df["signal"] = df["momentum"]
    
    # On negative signals, we go short.
    # On zero signal and lack of data set side to NA
    df["side"] = pd.NA
    
    df.loc[df["signal"] > 0, "side"] = "long"
    df.loc[df["signal"] < 0, "side"] = "short"

    # Max and min price wihtin the profit window will determine the profit for longs and shorts respective
    df["max_future_price"] = df["close"].rolling(number_of_look_forward_candles).max().shift(-number_of_look_forward_candles) # Get the max profit on the profit window, assuming take profit %
    df["min_future_price"] = df["close"].rolling(number_of_look_forward_candles).min().shift(-number_of_look_forward_candles) # Get the max profit on the profit window, assuming take profit %    
    
    df["profit"] = df["price_diff"]
    df["profit_max"] = df["profit"]
    df["profit_abs"] = df["profit_max"].abs()
    # Calculate profit separately for longs and shorts
    # using Pandas Mask
    # https://stackoverflow.com/a/33770421/315168
    #
    # We calculate both profit after X time,
    # and also max take profit, assuming
    # we could do a perfect trailing stop loss
    #
    #longs = (df["side"] == "long")
    #shorts = (df["side"] == "short")
    #df.loc[longs, "profit"] = df["price_diff"]
    #df.loc[shorts, "profit"] = -df["price_diff"]
    #df.loc[longs, "profit_max"] = (df["max_future_price"] - df["open"]) / df["open"]  # Get the profit based on max price
    #df.loc[shorts, "profit_max"] = -(df["min_future_price"] - df["open"]) / df["open"]  # Get the profit based on max price

    #df.loc[longs, "desc"] = df.agg('{0[ticker]} long'.format, axis=1)
    #df.loc[shorts, "desc"] = df.agg('{0[ticker]} short'.format, axis=1)

    df["profit"] = df["profit"].fillna(0)
    df["profit_max"] = df["profit_max"].fillna(0)

    # On too low trading volume we zero out signal
    candle_volume_threshold = daily_volume_threshold * (time_frame / pd.Timedelta(days=1))
    volume_threshold_exceeded = df["volume"] >= candle_volume_threshold
    df["signal"] = np.where(volume_threshold_exceeded, df["signal"], np.NaN)
    df["profit"] = np.where(volume_threshold_exceeded, df["profit"], np.NaN)
    df["profit_max"] = np.where(volume_threshold_exceeded, df["profit_max"], np.NaN)
    
    return df


def calculate_signal_vs_price_for_pair(
    grouped_candles: DataFrameGroupBy,
    pair: DEXPair
) -> pd.DataFrame:
    """Calculate signal vs. profit ratio for an individual pair."""
    try:
        df = grouped_candles.get_group(pair.pair_id).copy()
    except KeyError:
        # Scam pairs 
        return pd.DataFrame()
        
    df = calculate_signal_vs_profit(
        df,
        pair,
        lookback_window,
        profit_window,
        time_frame=time_bucket.to_pandas_timedelta(),
    )
    return df

print("Calculating signals")
grouped_candles = candles_df.groupby("pair_id")
all_pairs = tradeable_pairs_df.groupby("pair_id")
per_pair_data = [calculate_signal_vs_price_for_pair(grouped_candles, pair) for pair in pair_universe.iterate_pairs()]
df = pd.concat(per_pair_data)    

# Fix column order for table rendering
df.insert(0, 'signal', df.pop('signal'))
df.insert(0, 'profit', df.pop('profit'))


print(f"Total signals {len(df):,}")

### Display sample content for single trading pair

In [ ]:
# Display sample content for one pair
# Pick single pair to examine
# The first pair is ETH-USD
examined_pair = pair_universe.get_pair_by_human_description((ChainId.polygon, "uniswap-v3", "WMATIC", "USDC", 0.0005))
#examined_pair = pair_universe.get_pair_by_human_description((ChainId.polygon, "quickswap", "pSWAMP", "WMATIC"))
single_df = df.loc[df["pair_id"] == examined_pair.pair_id]

with pd.option_context('display.min_rows', 10), pd.option_context('display.max_rows', 10):
    display(single_df)


### Outlier filter

Get rid of abnormally large values caused on scam tokens and other issues in the input data.

In [ ]:
# Store for later
unfiltered_df = df

# Run outlier filter
#
# Use separate quantile for high and low because low min is -1.0 lose your all money when the token goes to zero in rug pull
#
# https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-a-pandas-dataframe
q_hi  = df["profit"].quantile(quantile_sanity_threshold)
q_low = df["profit"].quantile(1 - quantile_sanity_threshold)

#q_low = df["profit_max"].quantile(0.01)
#q_hi  = df["profit_max"].quantile(0.99)
df = df[(df["profit"] < q_hi) & (df["profit"] > q_low)]

print(f"Max profit high filtering {quantile_sanity_threshold}, quantile high is {q_hi},  quantile low is {q_low}")
#df = df[df["profit_abs"] < q_hi]

q_hi  = df["signal"].abs().quantile(quantile_sanity_threshold)
#q_low = df["profit_max"].quantile(0.01)
#q_hi  = df["profit_max"].quantile(0.99)
#df_filtered = df[(df["profit_max"] < q_hi) & (df["profit_max"] > q_low)]

#print(f"Max signal high filtering {quantile_sanity_threshold} quantile is {q_hi}")
#df = df[df["signal"].abs() < q_hi]

#price_df = df.sort_values('price_diff', ascending=False).drop_duplicates("timestamp")
#display(price_df)

### Best signal picker


In [ ]:
# Find maximum momentum df
profit_df = df.copy()
#profit_df.index = pd.MultiIndex.from_arrays([df.index, df["pair"]])
profit_df["signal"] = profit_df["signal"].fillna(0).replace([np.inf, -np.inf], 0)
profit_df["profit"] = profit_df["profit"].fillna(0).replace([np.inf, -np.inf], 0)
profit_df["profit_max"] = profit_df["profit_max"].fillna(0).replace([np.inf, -np.inf], 0)
profit_df["timestamp"] = profit_df.index
#display(profit_df)

#max_df = df.loc[df.groupby("timestamp")['signal'].idxmax()]
# max_df = df.groupby("timestamp")["profit"].max()
max_df = profit_df.sort_values('signal', key=abs, ascending=False).drop_duplicates("timestamp")
# max_df["signal"] = profit_df["signal"].fillna(0)

max_df = max_df.sort_values("profit", ascending=False)
print(f"Best picked results {len(max_df):,}")
# display(max_df)

#quantile_sanity_threshold = 0.90
#q_hi  = max_df["profit_abs"].quantile(quantile_sanity_threshold)
#print(f"Best profit high filtering {quantile_sanity_threshold} quantile is {q_hi}")
#max_df = max_df[max_df["profit_abs"] < q_hi]
#q_hi  = max_df["signal"].abs().quantile(quantile_sanity_threshold)
#print(f"Best signal high filtering {quantile_sanity_threshold} quantile is {q_hi}")
#max_df = max_df[max_df["signal"].abs() < q_hi]
#print(f"Best picked results, filtered {len(max_df):,}")

display(max_df)

# Example trading pair chart

Plot with our indicators



In [ ]:
import plotly.express as px
import plotly.graph_objects as go

from tradingstrategy.charting.candle_chart import visualise_ohlcv


fig = visualise_ohlcv(
    single_df,
    num_detached_indicators=3,
    subplot_names=["Price", "Signal", "Momentum", "PVT"],
)

fig.add_trace(
    go.Scatter(name="Signal", x=single_df.index, y=single_df["signal"]),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(name="Momentum", x=single_df.index, y=single_df["momentum"]),
    row=3,
    col=1,
)
fig.add_trace(
    go.Scatter(name="PVT", x=single_df.index, y=single_df["pvt"]),
    row=4,
    col=1,
)

fig.show()

# display(single_df)

## Examine signal

### EMA indicator correlation

See "Table 2: Correlation between the xk" in Momentum and trend following trading strategies for currencies and bitcoin.

In [ ]:
diff_df = pd.DataFrame()
for idx in range(len(short_long_ema_pairs)):
    diff_df[f"ema_diff_{idx+1}"] = df[f"ema_diff_{idx+1}"]
    
correlation_matrix = diff_df.corr()
display(correlation_matrix)

### Signal visualisation

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

from tradingstrategy.charting.candle_chart import visualise_ohlcv

fig = visualise_ohlcv(
    single_df,
    num_detached_indicators=1,
    subplot_names=["Price", "EMA diffs"],
)

# Plot all EMA diffs in a single chart
for idx, pair in enumerate(short_long_ema_pairs, start=1):
    short_ema, long_ema = pair
    fig.add_trace(
        go.Scatter(name=f"EMA diff #{idx} short:{short_ema} long:{long_ema}", x=single_df.index, y=single_df[f"ema_diff_{idx}"]),
        row=2,
        col=1,
    )

fig.show()


## Plot signal vs. price samples

Different scatter charts to examine if there is a correlation between the signal and the profit.

### Best signal picks

- For each time frame, pick the strongest signal

In [ ]:

signal_threshold = 0.10

fig = px.scatter(
    max_df, 
    x="signal", 
    y="profit", 
    trendline="ols", 
    title=f"Best signal per {signal_window_label}, profit window {profit_window_label}",
    trendline_color_override="black"
    #marginal_x="histogram", 
    #marginal_y="rug"    
)
fig.update_layout(legend=dict(title='Pair and direction'))
fig.show()

filtered_max_df = max_df[max_df["signal"].abs() >= signal_threshold]
fig = px.scatter(
    filtered_max_df, 
    x="signal", 
    y="profit", 
    trendline="ols", 
    title=f"Best signal per  {signal_window_label}, profit window {profit_window_label}, filtered for {signal_threshold}",
    trendline_color_override="black"
    #marginal_x="histogram", 
    #marginal_y="rug"    
)
fig.update_layout(legend=dict(title='Pair and direction'))
fig.show()


In [ ]:

#display(df)
#display(max_df)

fig = px.scatter(
    df, 
    x="signal", 
    y="profit_max", 
    trendline="ols", 
    # color="desc", 
    title=f"Max potential take profit, signal {signal_window_label}, profit window {profit_window_label}",
    #marginal_x="histogram", 
    #marginal_y="rug"
    trendline_color_override="black"
)
fig.update_layout(legend=dict(title='Pair and direction'))
fig.show()

fig = px.scatter(
    df, 
    x="signal", 
    y="profit", 
    trendline="ols", 
    title=f"Profit last day, signal {signal_window_label}, profit window {profit_window_label}",
    trendline_color_override="black"
    #marginal_x="histogram", 
    #marginal_y="rug"    
)
fig.update_layout(legend=dict(title='Pair and direction'))
fig.show()



### Long only vs. short 

Examine trade types separately to better see if one side is clearly superior.

In [ ]:
df_long_only = df.loc[df["side"] == "long"]

fig = px.scatter(
    df_long_only, 
    x="signal", 
    y="profit_max", 
    trendline="ols", 
    # color="desc", 
    trendline_color_override="black", 
    title=f"Long only max take profit, signal {signal_window_label}, profit window {profit_window_label}",

)
fig.show()

fig = px.scatter(
    df_long_only, 
    x="signal", 
    y="profit", 
    trendline="ols", 
    trendline_color_override="black", 
    title=f"Long only profit last day, signal {signal_window_label}, profit window {profit_window_label}"
)
fig.show()

fig = px.scatter(
    df_long_only, 
    x="signal", 
    y="profit_max", 
    trendline="ols", 
    trendline_color_override="black", 
    title=f"Long only max take profit, signal {signal_window_label}, profit window {profit_window_label}"
)
fig.show()

fig = px.scatter(
    df_long_only, 
    x="signal", 
    y="profit", 
    trendline="ols", 
    trendline_color_override="black", 
    title=f"Long only last profit, signal {signal_window_label}, profit window {profit_window_label}"
)
fig.show()


fig = px.scatter(
    df, 
    x="signal", 
    y="profit_max", 
    trendline="ols", 
    # color="desc", 
    title=f"Max potential take profit, signal {signal_window_label}, profit window {profit_window_label}",
    #marginal_x="histogram", 
    #marginal_y="rug"
    trendline_color_override="black"
)
fig.update_layout(legend=dict(title='Pair and direction'))
fig.show()



In [ ]:
df_short_only = df.loc[df["side"] == "short"]

max_df_short_only = max_df.loc[max_df["side"] == "short"]

fig = px.scatter(
    df_short_only, 
    x="signal", 
    y="profit_max", 
    trendline="ols", 
    title=f"Short only max take profit, signal {signal_window_label}, profit window {profit_window_label}",
    trendline_color_override="black",
)
fig.show()

fig = px.scatter(
    df_short_only, 
    x="signal", 
    y="profit", 
    trendline="ols",     
    title=f"Short only profit last day, signal {signal_window_label}, profit window {profit_window_label}",
    trendline_color_override="black",
)
fig.show()

fig = px.scatter(
    max_df_short_only, 
    x="signal", 
    y="profit", 
    trendline="ols",     
    title=f"Best short picks,signal {signal_window_label}, profit window {profit_window_label}",
    trendline_color_override="black",
)
fig.show()

### Filtered signal

Remove low signals below a certain threshold and see if it affects the correlation between the signal and the profit.

In [ ]:

threshold = 0.075

filtered_df = df_long_only.loc[abs(df_long_only["signal"]) >= threshold]

fig = px.scatter(
    filtered_df, 
    x="signal", 
    y="profit", 
    trendline="ols", 
    trendline_color_override="black", 
    title=f"Long-only filtered the signal at {threshold}, profit, signal {signal_window_label}, profit window {profit_window_label}"
)
fig.show()

fig = px.scatter(
    df_long_only, 
    x="signal", 
    y="profit", 
    trendline="ols", 
    trendline_color_override="black", 
    title=f"No filter, profit last day, signal {signal_window_label}, profit window {profit_window_label}")
fig.show()

filtered_df = df_short_only.loc[abs(df_short_only["signal"]) >= threshold]

fig = px.scatter(
    filtered_df, 
    x="signal", 
    y="profit_max", 
    trendline="ols", 
    trendline_color_override="black", 
    title=f"Short only filtered the signal at {threshold}, max take profit, signal {signal_window_label}, profit window {profit_window_label}"
)
fig.show()


## Heatmap

Invidual point cloud is hard to interpret, so render it as a heatmap instead.

In [ ]:

#clipped_df = df.loc[abs(df["signal"]) < 0.2]

fig = px.density_heatmap(
    df, 
    x="signal", 
    y="profit", 
    nbinsx=50, 
    title="Signal vs. profit"

)
fig.show()

#weth = df.loc[df["pair"] == "WETH-UDSC long"]
#fig = px.density_heatmap(df, x="signal", y="profit_max", nbinsx=30, title="WETH")
#fig.show()

## Hexbin plot

Try to make more sense from the point cloud.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
#import cmocean#  http://matplotlib.org/cmocean/

plt.figure(figsize=(7, 7))
ax = plt.hexbin(df["signal"], df["profit"], gridsize=50, mincnt=1) # cmocean.cm.algae is a cmocean colormap
# plt.set_title("Signal vs. Price matches")
plt.show()
# display(df)